# 課題（疎行列）

- すべての課題を実行せよ
- すべての課題が完了したら Jupyter の File メニューから Download as -> Markdown (.md) として結果をダウンロードし， Bb9 の「課題」からファイル添付で提出せよ

In [ ]:
# 初期化なので最初に実行してください
include("../lib/SparseMatrix.jl")
using LinearAlgebra
using .SparseMatrix
using Test
using Printf

**注意**: この演習で使う SparseMatrix パッケージは自作パッケージなので無保証です．一般の計算で疎行列を使う場合は using SparseArrays として Julia が提供しているものを利用してください．ただし，現時点で SparseArrays には CSC しかないです

## 課題1
- 疎行列をあつかうデータ構造 CSR, CSC, COO を確認し 5 x 5 の行列のとき， 変数配列の大きさをそれぞれ求め必要なバイト数を答えよ

In [ ]:
# 疎行列を作成
A = sprand(5, 5, 0.5, Matrix)
display(A)
println(typeof(A))

### 課題1-1: A で使っているバイト数を計算

- 以下の表の？を埋めよ

| 変数 | 要素数 | 型 | 一つの要素に必要なバイト数 | バイト数 |
|:---:|:---:|:---:|:---:|:---:|
| A | ? | Flotat64 | ? | ? |
| 合計| --- | --- | --- | ? |

In [ ]:
# CSRデータ構造
csr = SparseCSR(A)
println(typeof(csr))
println(csr.rowptr)
println(csr.colind)
println(csr.val)

### 課題1-2: `csr` で使っているバイト数を計算

| 変数 | 要素数 | 型 | 一つの要素に必要なバイト数 | バイト数 |
|:---:|:---:|:---:|:---:|:---:|
| rowptr | ? | Int64 | ? | ? |
| colind | ? | Int64 | ? | ? |
| val | ? | Flotat64 | ? | ? |
| 合計| --- | --- | --- | ? |

In [ ]:
# CSCデータ構造
csc = SparseCSC(A)
println(typeof(csc))
println(csc.colptr)
println(csc.rowind)
println(csc.val)

### 課題1-3: `csc` で使っているバイト数を計算

| 変数 | 要素数 | 型 | 一つの要素に必要なバイト数 | バイト数 |
|:---:|:---:|:---:|:---:|:---:|
| colptr | ? | Int64 | ? | ? |
| rowind | ? | Int64 | ? | ? |
| val | ? | Flotat64 | ? | ? |
| 合計| --- | --- | --- | ? |

In [ ]:
# COOデータ構造
coo = SparseCOO(A)
println(typeof(coo))
println(coo.rowind)
println(coo.colind)
println(coo.val)

### 課題1-4: `coo` で使っているバイト数を計算

| 変数 | 要素数 | 型 | 一つの要素に必要なバイト数 | バイト数 |
|:---:|:---:|:---:|:---:|:---:|
| rowind | ? | Int64 | ? | ? |
| colind | ? | Int64 | ? | ? |
| val | ? | Flotat64 | ? | ? |
| 合計| --- | --- | --- | ? |

### 課題1-5
- CSR, CSC, COO それぞれで n x n 行列で必要なバイト数が密行列より小さくなる最大の要素数を求めよ

| n | データフォーマット | 要素数 |
|:---:|:---:|:---:|
| n=5 | CSR | ? |
| n=5 | CSC | ? |
| n=5 | COO | ? |
| n=10 | CSR | ? |
| n=10 | CSC | ? |
| n=10 | COO | ? |

## 課題2
- 行列・ベクトル積の計算量を求める

### 課題2-1
- 下記の CSR用，CSC用，COO用のspmul を完成させよ

### spmul の説明
- 引数
    - A: m行n列の疎行列
    - x: n 要素のベクトル
- 戻り値
    - A * x の結果
- 関数内の変数
    - m: 行数
    - n: 列数
    - i: 行のインデックスを示す
    - j: 列のインデックスを示す
    - z: 何番目の非零要素かを示す

In [ ]:
# CSR用
function spmul(A::SparseCSR, x)
    m, n = size(A)
    y = zeros(m)
    for i = 1:m
        for z = A.rowptr[i]:A.rowptr[i+1]-1
            j = A.colind[z]
            # ここに適切なコードを記述
        end
    end
    return y
end

In [ ]:
# CSC用
function spmul(A::SparseCSC, x)
    m, n = size(A)
    y = zeros(m)
    for j = 1:n
        for z = A.colptr[j]:A.colptr[j+1]-1
            i = A.rowind[z]
            # ここに適切なコードを記述
        end
    end
    return y
end

In [ ]:
# COO用
function spmul(A::SparseCOO, x)
    m, n = size(A)
    y = zeros(m)
    for z = 1:nnz(A)
        i = A.rowind[z]
        j = A.colind[z]
       # ここに適切なコードを記述
    end
    return y
end

#### spmulのテストプログラム
- 作成できたら以下のテストプログラムを実行してすべて**Test Passed**になることを確認
- **Test Failed** が出たら結果が計算が間違っているので修正

In [ ]:
for k = 1:10
    m = rand(1:10)
    n = rand(1:10)
    p = rand()
    t = rand([SparseCSR, SparseCSC, SparseCOO])
    A = sprandn(m, n, p, t)
    x = rand(n)
    println(@test spmul(A, x) == A * x)
end

### 課題2-1
- 以下の行列Aとベクトルxの積 Ax を行ったとき，spmul 内での乗除算，加減算の回数を数えよ
$$
A = \begin{pmatrix}
1.0 & 0.0 & 0.0 \\
0.0 & 1.0 & 1.0 \\
1.0 & 1.0 & 1.0 \\
0.0 & 1.0 & 0.0 \\
1.0 & 0.0 & 1.0 \\
\end{pmatrix}, \quad
x = \begin{pmatrix}
1.0 \\
0.0 \\
1.0
\end{pmatrix}.
$$

### 課題2-1の解答

| フォーマット | 乗除算 | 加減算 |
|:-------:|:-------:|:-------:|
| 密行列 | ? | ? |
| CSR |   ?     |     ?    |
| CSR |   ?     |     ?    |
| CSC |   ?     |     ?    |
| COO |   ?     |     ?    |

# 課題3
- power method を行い密行列と疎行列の速度を比較する

### 課題3-1（前にやった課題ですがもう一度やってください）
- べき乗法によって固有値を求める関数 powermethod を作成せよ
- 関数が作成できたらテストを実行して **Test Passed** になることを確認せよ
- 関数がきちんとできていない場合 **Test Failed** が出力されるので **Test Passed**になるように修正せよ

### powermethodの説明（再掲）
- 引数
    - A: 正方行列（2次元配列）
- 戻り値
    - lambda1: 最大固有値
    - x: 固有ベクトル
- 関数内の変数
    - lambda1, lambda2: 固有値
    - err: 固有値の相対誤差

In [ ]:
function powermethod(A)
    x = fill(1.0, size(A)[1])
    err = 1.0
    lambda1 = dot(x, A * x) / dot(x,x)
    while err > 1.0e-10
        # コードを記述
        # (1) Ax として x を更新
        # (2) x を norm(x) で正規化
        lambda2 = dot(x, A * x) / dot(x,x)
        err = abs(lambda2 - lambda1)/abs(lambda1)
        lambda1 = lambda2
    end
    (lambda1, x)
end

#### powermethodのテストプログラム
- 以下のテストプログラムを実行してすべて**Test Passed**になることを確認
- **Test Failed** が出たら結果が計算が間違っているので関数を修正

In [ ]:
for k = 1:10
    s = rand(1:10)
    A = rand(s, s)
    A = A * A' # 固有値をすべて実数にするため
    lambda, x = powermethod(A)
    maxeigen = eigmax(A)
    println(@test  maxeigen ≈ lambda)
end

### 課題3-2
- 密行列，疎行列それぞれで最大固有値を得るまでの実行時間を計測し，速度に関する考察を行え

### genmatrixの説明
- 概要
    - 固有値が実数の 100 x 100 の密行列とそれに対するCSRフォーマットの行列をランダムに生成する
- 引数
    - p: 非零要素のおおよそ割合（0 から 1 の間の値）
- 戻り値
    - (d, A, spa)
        - d: 非零要素の密度（パーセンテージ）
        - A: 密行列
        - spA: 疎行列（CSRフォーマット）
- 関数内の変数
    - G: 100 x 100 の行列

In [ ]:
# 行列を作成し，非零要素の密度を測定
function genmatrix(p)
    G=sprandn(100, 100, p, Matrix)
    A = G' * G # 固有値を実数にするため
    spA = SparseCSR(A)
    (nnz(spA)/length(A)*100, A, spA) # (密度，密行列，疎行列)
end

In [ ]:
# 実験（必ず二回実行してください）
for p in 0.01:0.01:0.2
    d, A, spA = genmatrix(p)
    y1 = @elapsed(powermethod(A))
    y2 = @elapsed(powermethod(spA))
    @printf("密度　%.2f%%\t密行列 %f\t疎行列　%f\n", d, y1, y2)
end

### 考察

ここに考察を記入してください